In [ ]:
import ase
import ase.io
import ase.visualize
import numpy as np

import matplotlib.pyplot as plt

import tb_mean_field_hubbard as tbmfh

In [ ]:
# GEOMETRY
geom = ase.io.read("geom/clars_goblet.xyz")

# Spin guess is set in atom tags (1 - up, 2 - down)
# one option to modify it is with the following (edit->modify & file->save):
#ase.visualize.view(geom)

In [ ]:
# CALCULATION PARAMETERS

t1 = 2.7 # first nearest neighbor
t2 = 0.1 # second nearest neighbor
t3 = 0.4 # third nearest neighbor

t_list = [t1, t2, t3]

charge = 0
multiplicity = 1

In [ ]:
mfh_model = tbmfh.MeanFieldHubbardModel(geom, t_list, charge, multiplicity)

mfh_model.print_parameters()

mfh_model.visualize_spin_guess()
#mfh_model.visualize_tb_model()

In [ ]:
mfh_model.run_mfh(u = 3.0, print_iter=False, plot=False)

In [ ]:
mfh_model.report(num_orb=2)

In [ ]:
# Visualize one specific orbital

spin = 0
index = 0
print("Orbital index: %d, spin %d" % (index, spin))
print("Energy: %.6f eV" % mfh_model.evals[spin][index])
mfh_model.plot_orbital(mo_index=index, spin=spin)

# corresponding eigenvector (each element corresponds in order to atoms defined in geom/mfh_model.ase_geom)
evec = mfh_model.evecs[spin][index]

# Series calculations

In [ ]:
geom = ase.io.read("geom/clars_goblet.xyz")

t = 2.7

singlet_mfh_model = tbmfh.MeanFieldHubbardModel(geom, [t], charge=0, multiplicity=1)
triplet_mfh_model = tbmfh.MeanFieldHubbardModel(geom, [t], charge=0, multiplicity=3)
# (for the clar's goblet, the correct triplet state is found also with singlet initial guess)

In [ ]:
# Reproducing Extended Data Fig. 1 from
# Mishra 2020 "Topological frustration induces unconventional magnetism in a nanographene"

u_t_ratios = np.arange(0.5, 1.6, 0.1)

singlet_energies = []
triplet_energies = []

for ut_ratio in u_t_ratios:
    
    u = ut_ratio * t
    
    singlet_mfh_model.run_mfh(u)
    singlet_energies.append(singlet_mfh_model.energy)
    
    triplet_mfh_model.run_mfh(u)
    triplet_energies.append(triplet_mfh_model.energy)
    
singlet_energies = np.array(singlet_energies)
triplet_energies = np.array(triplet_energies)

st_gap = triplet_energies - singlet_energies

In [ ]:
plt.plot(u_t_ratios, st_gap*1000, 'o-')
plt.ylabel("triplet - singlet [meV]")
plt.xlabel("U/t")
plt.show()